In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# To make the text in plots easier to read:
font = {'family' : 'Arial',
        'size'   : 18}
import matplotlib
matplotlib.rc('font', **font)

In [ ]:
def coarsen(f):
    return f[1::2]  # This slicing just takes the odd-numbered points

def interpolate(f,alpha,beta):
    m_coarse=len(f)
    m_fine  =2*m_coarse+1
    f_interp = np.zeros(m_fine)
    f_interp[1::2]=f                          #Set the values of the odd numbered points
    f_interp[2:-1:2]=0.5*(f[:-1]+f[1:])       #Set the values of the (interior) even numbered points
    f_interp[0]=0.5*(f_interp[1]+alpha)     #Set the values of the endpoints
    f_interp[-1]=0.5*(f_interp[-2]+beta)
    return f_interp

def Jacobi(U,f,alpha,beta,m,nu,omega=2./3):
    """Perform nu Jacobi iterations on a grid with m points, with initial guess U, right hand side function f and 
       Dirichlet boundary conditions with values alpha and beta.  The function returns both the approximate
       solution and the residual."""
    h=1./(m+1)
    F=0.5*h**2*f.copy()
    F[0]-=alpha/2.; F[-1]-=beta/2.
    e=np.ones(m-1)
    G=0.5*(np.diag(e,-1)+np.diag(e,1))
    for i in range(nu):
        U=(1.-omega)*U + omega*(np.dot(G,U)-F)
    A=2./h**2*(G-np.eye(m))
    FF=f.copy(); FF[0]-=alpha/h**2; FF[-1]-=beta/h**2
    rr=FF-np.dot(A,U)
    return U,rr

In [ ]:
phi = lambda x: 20.* np.pi * x**3
f = lambda x: -20 + 0.5*120*np.pi*x * np.cos(phi(x)) - 0.5*(60*np.pi*x**2)**2 * np.sin(phi(x))
    
def Vcycle(k,nu,omega,smooth_after_interp=True):
    m=2**k-1
    rdep=k-1  # Recursion depth; this is how many grids down we want to go
              # rdep=k-1 gives a full V-cycle
    alpha=1.; beta=3.
    U=np.linspace(alpha,beta,m)  # Initial guess
    x=np.linspace(0,1,m+2); x=x[1:-1]  # grid
    u = 1.+12.*x-10.*x**2 + 0.5*np.sin(phi(x))

    F=f(x)
    r=[None]*(rdep+1); error=[None]*(rdep+1) # This just initializes these lists to have the right length
    U,rr=Jacobi(U,F,alpha,beta,m,nu,omega)  # Initial iteration on fine grid
    for i in range(1,rdep+1): # Going down
        m=(m-1)//2 #  = 2**(k-i) - 1
        r[i]=coarsen(rr)             # residual restricted to next coarser grid
        error[i],rr=Jacobi(np.zeros(m),-r[i],0.,0.,m,nu,omega)
    for i in range(1,rdep): # Coming up
        m=2*m+1
        err=error[rdep-i]-interpolate(error[rdep+1-i],0,0)     # Interpolate and subtract the correction
        if smooth_after_interp:
            error[-i-1],rr=Jacobi(err,-r[rdep-i],0.,0.,m,nu,omega)
        else:
            error[-i-1] = err

    m=2*m+1
    U=U-interpolate(error[1],0,0)                   # final solution correction
    U,rr=Jacobi(U,F,alpha,beta,m,nu,omega)                #Final iterations on original grid
    
    fig, ax = plt.subplots(figsize=(12,8),subplot_kw={'facecolor':'#EEEEEE',
                                       'axisbelow':True})
    ax = plt.axes(xlim=(0, 1), ylim=(-5,6))
    ax.plot(x,u,x,U,'o-',x,U-u);
    plt.legend(['Exact','Jacobi','Error'],loc='best')
    #return fig

interact(Vcycle, 
               k=widgets.IntSlider(min=5,max=10,value=8), 
               nu=widgets.IntSlider(min=1,max=6,value=1,name='$\nu$'),
               omega=widgets.FloatSlider(min=0,max=1,step=0.05,value=2./3),
               smooth_after_interp=widgets.Checkbox(name='Smooth after interpolation'));

In [ ]:
Vcycle(8,3,2./3)